# Deep Learning Aplicado a la Detección Automática de Valvulopatías Usando Sonidos Cardíacos
John Jairo Gelpud Chachinoy<sup>1</sup>, Silvia Gabriela Castillo Pastuzan<sup>2</sup>

Director: PhD. Wilson O. Achicanoy Martínez<sup>3</sup>

Coasesor: PhD(c). Mario Fernando Jojoa<sup>4</sup>

<sup>1</sup>Departamento de Electrónica, Universidad de Nariño, Pasto, e-mail: johnjairog@udenar.edu.co

<sup>2</sup>Departamento de Electrónica, Universidad de Nariño, Pasto, e-mail:gabrielacast@udenar.edu.co

<sup>3</sup>Departamento de Electrónica, Universidad de Nariño, Pasto, e-mail:wilachic@udenar.edu.co

<sup>4</sup>Universidad de Deusto, España, e-mail:mariojojoa@deusto.es


# Introducción
Este código hace parte de los documentos desarrollados como parte del trabajo de tesis de grado para optar por el título de ingeniero electrónico en la Universidad de Nariño. El código desarrollado permite extraer vectores de características a partir de imágenes de escalogramas usando la técnica de Transfer Learning y las redes neuronales convolucionales (CNN) ResNet152 y VGG16.

## Código de Inicialización

### Importar Librerías
Importa todas las librerías necesarias para ejecutar el código.

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import zipfile
import io
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score
import scikitplot as skplt

plt.style.use('seaborn-white')
plt.rcParams.update({'font.size':18})

cudnn.benchmark = True
plt.ion()   # interactive mode

### Cargar Datos
El conjunto de datos que se va a utilizar para entrenar y evaluar los modelos de Deep Learning debe tener la estructura que se muestra en la Fig. 1..

![Figure 1](https://github.com/johnjairo04/Deep-Learning-for-Heart-Valve-Disease-Detection/blob/master/Codes/Python/Classification%20Algorithms/dataset.svg?raw=1)
**Figura 1 | Estructura del conjunto de datos.**

El conjunto de datos se puede cargar de dos formas dependiendo del entorno en el que se esté trabajando.

- **Google Colab.** Si el código se está ejecutando en Google Colab el conjunto de datos debe ser un archivo comprimido zip que contenga las carpetas: "train", "val", y "test". En cada subconjunto de datos deben haber dos carpetas, una para cada categoria.
- **Sistema local.** Si el código se está ejecutando localmente se debe especificar el directorio donde se encuentran los subconjuntos "train", "val", y "test".

In [17]:
# Si se está ejecutando en Google Colab definir colab=True
colab = False
# Si se está ejecutando localmente especificar el directorio en dataset_path
dataset_path = 'D:/UDENAR/Electronic Engineering/Trabajo de Grado/Results/Teager/Classification/CardioDeep2.1/'
# Directorio donde se van a almacenar los vectores de características extraídos
features_path = 'C:/Users/usuario/OneDrive/Electronic Engineering/Trabajo de Grado/Codes/Python/version 2.0/CardioDeep2.1/'

if colab:
    dataset_path = 'Datasets/'
    from google.colab import files
    uploaded = files.upload()
    data = zipfile.ZipFile(io.BytesIO(uploaded['Datasets.zip']), 'r')
    data.extractall()
else:
    print('Ejecutando en sistema local')

Ejecutando en sistema local


In [3]:
# Normalización de las imágenes
data_transforms = {
    'train':transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val':transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# Crea los datasets y dataloaders con un batch de 1 imagen, y aplica las transformaciones
image_datasets = {x: datasets.ImageFolder(dataset_path+x, data_transforms[x])
                      for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1, shuffle=False,
                                              num_workers=2) for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes # nombres de las categorias
# Selecciona la GPU o CPU como dispositivo para realizar las operaciones y
# almacenar las variables
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Extracción de Características

### Funciones
Las siguientes funciones permiten extraer las características de las imágenes y almacenarlas en un archivo .xlsx con su respectiva etiqueta (0=Anormal, 1=Normal)

In [5]:
def feature_extraction(model, dataloaders, dataset_sizes, num_ftrs):
    # Obtiene las características de las imágenes de entrenamiento, validación y prueba, y devuelve un arreglo con los vectores de
    # características y sus respectivas etiquetas
    train_ftrs = torch.empty((dataset_sizes['train'], num_ftrs+1), device='cpu') # almacena las características de las imágenes
    # de entrenamiento
    val_ftrs = torch.empty((dataset_sizes['val'], num_ftrs+1), device='cpu') # almacena las características de las imágenes de
    # validación
    test_ftrs = torch.empty((dataset_sizes['test'], num_ftrs+1), device='cpu') # almacena las características de las imágenes
    # de prueba
    
    # Iterar sobre cada subconjunto de imágenes
    for dataset, array in zip(['train', 'val', 'test'], [train_ftrs, val_ftrs, test_ftrs]):
        
        # Iterar sobre cada imagen
        for i, data in zip(range(dataset_sizes[dataset]), dataloaders[dataset]):
            inputs = data[0] # imagen
            labels = data[1] # etiqueta de la imagen
            inputs = inputs.to(device) # mueve los tensores a la GPU o CPU
            labels = labels.to(device) # mueve los tensores a la GPU o CPU
            outputs = model(inputs) # salidas del modelo
            array[i, :-1] = outputs.squeeze() # vector de características
            array[i, -1] = labels # guarda la etiqueta en la última posición del vector de características  
    
    train_ftrs = train_ftrs.cpu().detach().numpy()
    val_ftrs = val_ftrs.cpu().detach().numpy()
    test_ftrs = test_ftrs.cpu().detach().numpy()
    
    return train_ftrs, val_ftrs, test_ftrs

In [6]:
def save_features(train_ftrs, val_ftrs, test_ftrs, model, path):
    # Guarda las características extraídas y sus etiquetas en un archivo .xlsx.
    columns = ['ftr_'+str(x) for x in range(train_ftrs.shape[1]-1)] # crea una columna para cada característica
    columns.append('label') # crea una columna para la etiqueta
    
    train_dataset = pd.DataFrame(train_ftrs, columns=columns) # convierte a DataFrame el arreglo que contiene
    # las características y etiquetas del subconjunto de entrenamiento
    val_dataset = pd.DataFrame(val_ftrs, columns=columns) # convierte a DataFrame el arreglo que contiene
    # las características y etiquetas del subconjunto de validación
    test_dataset = pd.DataFrame(test_ftrs, columns=columns) # convierte a DataFrame el arreglo que contiene
    # las características y etiquetas del subconjunto de prueba
    
    # Convierte cada DataFrame a un archivo .xlsx.
    for phase, dataframe in zip(['train', 'val', 'test'], [train_dataset, val_dataset, test_dataset]):
        dataframe.to_excel(path+model+'/'+phase+'.xlsx')
    
    return train_dataset, val_dataset, test_dataset

### CNNs como extractores de características

#### ResNet152

In [7]:
res_model = models.resnet152(pretrained=True)
res_num_ftrs = res_model.fc.in_features

modules = list(res_model.children())[:-1]
fe_res_model = nn.Sequential(*modules)

for param in res_model.parameters():
    param.requires_grad = False

fe_res_model.eval()
fe_res_model.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [8]:
res_train_ftrs, res_val_ftrs, res_test_ftrs = feature_extraction(fe_res_model, dataloaders,
                                                                 dataset_sizes, res_num_ftrs)

In [18]:
res_train_df, res_val_df, res_test_df = save_features(res_train_ftrs, res_val_ftrs,
                                                      res_test_ftrs, 'ResNet152',
                                                      features_path)

#### VGG16

In [19]:
vgg_model = models.vgg16(pretrained=True)
vgg_num_ftrs = list(vgg_model.classifier.children())[-1].in_features

fe_vgg_model = vgg_model
fe_vgg_model.classifier = fe_vgg_model.classifier[:-1]

for param in fe_vgg_model.parameters():
    param.requires_grad = False
    
fe_vgg_model.eval()
fe_vgg_model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
vgg_train_ftrs, vgg_val_ftrs, vgg_test_ftrs = feature_extraction(fe_vgg_model, dataloaders,
                                                                 dataset_sizes, vgg_num_ftrs)

In [21]:
vgg_train_df, vgg_val_df, vgg_test_df = save_features(vgg_train_ftrs, vgg_val_ftrs,
                                                      vgg_test_ftrs, 'VGG16',
                                                      features_path)

In [22]:
torch.cuda.empty_cache()